# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv_path = "../WeatherPy/citydf.csv"

# Import the books.csv file as a DataFrame
weather_df = pd.read_csv(csv_path, encoding="utf-8")
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,barrow,71.2906,-156.7887,10.40,79,1,6.69,US,1619445931
1,waspan,14.7419,-83.9717,80.60,83,20,3.09,NI,1619446125
2,pangnirtung,66.1451,-65.7125,37.40,87,90,3.09,CA,1619446126
3,avarua,-21.2078,-159.7750,69.80,100,54,1.03,CK,1619446127
4,yerbogachen,61.2767,108.0108,19.83,73,91,4.65,RU,1619446128


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:

weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,barrow,71.2906,-156.7887,10.40,79,1,6.69,US,1619445931
1,waspan,14.7419,-83.9717,80.60,83,20,3.09,NI,1619446125
2,pangnirtung,66.1451,-65.7125,37.40,87,90,3.09,CA,1619446126
3,avarua,-21.2078,-159.7750,69.80,100,54,1.03,CK,1619446127
4,yerbogachen,61.2767,108.0108,19.83,73,91,4.65,RU,1619446128


In [5]:
locations = weather_df[["Lat", "Lng"]]

In [6]:
weight  = weather_df["Humidity"].astype(float)

In [7]:
figW = gmaps.figure()

# Create heat layer
heat_layerW = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
figW.add_layer(heat_layerW)

# Display figure
figW

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
#  * A max temperature lower than 90 degrees but higher than 70.

#   * Wind speed less than 20 mph.

#   * less than 30 cloudiness.

In [8]:
Weather_Criteria = weather_df[(weather_df["Max Temp"]>70)&(weather_df["Max Temp"]<90)& (weather_df["Wind Speed"]<20)& (weather_df["Cloudiness"]<10)]
WC= Weather_Criteria.dropna()
WC

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,keti bandar,24.1447,67.4497,83.88,70,0,6.82,PK,1619446128
38,rikitea,-34.4187,19.2345,72.00,61,6,0.89,ZA,1619446141
66,arman,40.4272,71.7189,80.60,19,0,2.06,UZ,1619446153
83,almonte,-22.0920,-70.1979,70.77,58,0,3.09,CL,1619446161
88,krasnoselkup,17.1561,-89.0714,78.80,78,0,3.09,BZ,1619446164
107,margate,-33.0153,27.9116,77.00,61,0,3.60,ZA,1619445908
115,mar del plata,19.1059,37.3321,89.78,53,3,4.20,SD,1619446174
154,samusu,10.5669,72.6420,85.37,68,0,2.39,IN,1619446195
156,biloela,13.5928,-89.8275,77.00,83,0,1.16,SV,1619446150
172,kavaratti,-20.3297,-40.2925,84.20,70,0,3.60,BR,1619446115


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
import requests
import json

In [23]:
hotel_df = pd.read_csv("../WeatherPy/citydf.csv")
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,barrow,71.2906,-156.7887,10.40,79,1,6.69,US,1619445931
1,waspan,14.7419,-83.9717,80.60,83,20,3.09,NI,1619446125
2,pangnirtung,66.1451,-65.7125,37.40,87,90,3.09,CA,1619446126
3,avarua,-21.2078,-159.7750,69.80,100,54,1.03,CK,1619446127
4,yerbogachen,61.2767,108.0108,19.83,73,91,4.65,RU,1619446128


In [21]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'Hotel Name'

In [24]:
hotel_df["Lat"] = ""
hotel_df["Lng"] = ""
hotel_df["Hotel Name"] = ""
hotel_df["Hotel Address"] = ""
hotel_df["Hotel Rating"] = ""
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Address,Hotel Rating
0,barrow,,,10.40,79,1,6.69,US,1619445931,,,
1,waspan,,,80.60,83,20,3.09,NI,1619446125,,,
2,pangnirtung,,,37.40,87,90,3.09,CA,1619446126,,,
3,avarua,,,69.80,100,54,1.03,CK,1619446127,,,
4,yerbogachen,,,19.83,73,91,4.65,RU,1619446128,,,


In [29]:
# create a params dict that will be updated with new city each iteration
params = {"key": g_key}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in hotel_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    city = row['City']
    country = row['Country']

    # update address key value
    params['address'] = f"{city},{country}"

    # make request
    hotels_lat_lng = requests.get(base_url, params=params)
    
    # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
    # print(cities_lat_lng.url)
    
    # convert to json
    hotels_lat_lng = hotels_lat_lng.json()

    hotel_df.loc[index, "Lat"] = hotels_lat_lng["results"][0]["geometry"]["location"]["lat"]
    hotel_df.loc[index, "Lng"] = hotels_lat_lng["results"][0]["geometry"]["location"]["lon"]

IndexError: list index out of range

In [19]:
# params dictionary to update each iteration

params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in WC.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
   
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    locations = requests.get(base_url, params=params).json()
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = locations["results"][0]["name"]
        print(locations)
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.
{'html_attributions': [], 'next_page_token': 'ATtYBwIs5ISsdbV4wEOON2fEJ7S3D_nBqi08E8GjqpRgo-Lt7q2sEvpm7px8_Cejfj6wrah-a2YmqQMiy8xOIo6Cs8YdKhy64Enl2bLlVZ71BxIkv3sVVjT7EGnVItz9VlmZKc7tGIUrNlu2HI7mP68IPpqom_DDhIUMi-_dGfU6Et8XCgFDroqd4cXE6A67jhI0sO4Zjf1WpcFha7eBqsaYyWRWpXMUaxzNNVwesFKYrYstMFGSs_8agOnr6aZo9p5ANrWSBFtuX1GiqUWkWt_6wEmMktk2MuYh6SbuanKiNqQM2wWx_jswmGoB7we_aPTEeNrMQHr6lvCmSjoumbJ-JB_Id-FF1imWwWpkIU9T3kKoJIB17gguehhcTqz-s-sKEXXe0K7lkkYdYZyQlWwgvVXfr__Uodx3E556WPyb4pc_eB4UOVPaGnEIKH9OEbIVJg', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -34.4211674, 'lng': 19.2431805}, 'viewport': {'northeast': {'lat': -34.41974982010728, 'lng': 19.24449872989272}, 'southwest': {'lat': -34.42244947989273, 'lng': 19.24179907010728}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Harbour House Hotel', 'opening_hours': {'open_now': True}, 'photos': [{'height': 4000, 'html_attribution

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 40.4416845, 'lng': 71.71644549999999}, 'viewport': {'northeast': {'lat': 40.44298927989273, 'lng': 71.71764267989273}, 'southwest': {'lat': 40.44028962010729, 'lng': 71.71494302010728}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Diyor hotel', 'opening_hours': {'open_now': True}, 'photos': [{'height': 3000, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/105019193117127251399">A Google User</a>'], 'photo_reference': 'ATtYBwLVrgiYaBW8Exkv2mz8UX40h-InxKjGB1b-RtHv5DHalur4jWI8IYxce3H24rdcD_A9VA1NbqoP1-BOiXhlVVv3gfMQhjP_6v2Axc1-Lz8E3nWwxU_jH2e9-e-aYxG-HDm-SCALCmCuo4dCY75RXAIG2gvz_90TyNwsnj-GIJiQ4Fjq', 'width': 4000}], 'place_id': 'ChIJjRuZUaJ3uzgRwChDsDVzDRY', 'plus_code': {'compound_code': 'CPR8+MH Margilan, Uzbekistan', 'global_code': '8JGHCPR8+MH'}, 'rating': 4.9, 'reference': 'ChIJjRuZUaJ3uzgRwChDsDVzDRY', 'sc

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -22.0800553, 'lng': -70.1932312}, 'viewport': {'northeast': {'lat': -22.07873567010728, 'lng': -70.19185232010727}, 'southwest': {'lat': -22.08143532989273, 'lng': -70.19455197989272}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Hostal Paraiso', 'opening_hours': {'open_now': True}, 'photos': [{'height': 1072, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/113153287382081622062">A Google User</a>'], 'photo_reference': 'ATtYBwJbDUVpE_TuA4ge1zJ-KszhXSFws98eXc6fp-nBpHLOshMTUfLmOSyRCvA1T5lnqwa5ME1JE0wKEbcFxmkkJGBmWlDkxXzDXUbBOC1h95uv4VTIyYtSDY8L2MsctqH4s8FvGq18D_n8s9PlUewVtXDtTEl8qdIJkRmur1-lqdeBemup', 'width': 1073}], 'place_id': 'ChIJ1SeMpwJ3rZYRJb6AbazTexQ', 'plus_code': {'compound_code': 'WR94+XP Tocopilla, Chile', 'global_code': '579FWR94+XP'}, 'rating': 4.3, 'reference': 'ChIJ1SeMpwJ3rZYRJb6AbazTexQ', 'scop

{'html_attributions': [], 'next_page_token': 'ATtYBwI3DlxgGwJNhzXSxAAeV7yFWca53sqJhsW_HaB6CHMxWtfkJ3KFkWhO26JnnkuIMJqDUTURLTfOYPvmDCpZo0ycYvk5LBc4954ds1rMgbPh6QoCbF5djXBnXT4JX5uvlUiOvUBQcPdAz_uFie4oLC8cXZkxsjn2EAgB0-a9abDt3F3Gj-8w_cwtz_BrhCnycv-5HefpP9qd7Twjn2nMzeMtEB6GJw9Az6SabtrM9MlPptHbR5UJrxSAUyLJTyEOhT5wolqsaB_tB9WRhwYvNuCvQd983EteoLIQJ8__scNClDcOP_akEcjYy3TDJNGQMoRFINHhLRq0NokCfpsml7sfKf9Vhvrs4bnwxJotn-3NhKNcYY18J1PfdGIJpqxS86UFLCPTSrw2SMCdT2nJD1PrnC5JxpZRrwwgs2RZeAgYv417YsvVzYjH_ij3GeLw-Q', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -32.9834715, 'lng': 27.9510976}, 'viewport': {'northeast': {'lat': -32.98171747010728, 'lng': 27.95267107989272}, 'southwest': {'lat': -32.98441712989272, 'lng': 27.94997142010727}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Blue Lagoon Hotel', 'photos': [{'height': 4128, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/106228235576076990296

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 19.0989824, 'lng': 37.333168}, 'viewport': {'northeast': {'lat': 19.10015867989272, 'lng': 37.33456162989272}, 'southwest': {'lat': 19.09745902010728, 'lng': 37.33186197010728}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'فندق عرب بلازا', 'place_id': 'ChIJVfBllq7X2RURel6yfnZTBNM', 'plus_code': {'compound_code': '38XM+H7 Suakin, Sudan', 'global_code': '7GFV38XM+H7'}, 'rating': 0, 'reference': 'ChIJVfBllq7X2RURel6yfnZTBNM', 'scope': 'GOOGLE', 'types': ['lodging', 'point_of_interest', 'establishment'], 'user_ratings_total': 0, 'vicinity': 'سواكن'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 19.0990772, 'lng': 37.3331595}, 'viewport': {'northeast': {'lat': 19.10045862989272, 'lng': 37.33432852989272}, 'southwest': {'lat': 19.09775897010728, 'lng': 37.33162887010727}}}, 'icon': 'https://maps.gstatic.com/mapfil

...

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -21.7966272, 'lng': -54.53920859999999}, 'viewport': {'northeast': {'lat': -21.79537842010728, 'lng': -54.53793997010727}, 'southwest': {'lat': -21.79807807989272, 'lng': -54.54063962989272}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Gold Hotel', 'photos': [{'height': 2592, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/108169819907451892045">Amós Araujo</a>'], 'photo_reference': 'ATtYBwLNlAuHvqNwQ7KdYub1eqIwgUMoM1i-SKidcnMSNyeCCY-Tf_sa5_ZyHGFZ2ENAbpTrlqS95GHgbvMBR-5808dpjR8TLsFY10tim6rbHJEESl5LyC4OOviXxd3E9UEePNmFuVjhBD7Zr339nLajDaFtBP7W9hxK8ROXdp2I8iOrS13H', 'width': 4608}], 'place_id': 'ChIJcW5xkbzuiJQRAi8Y7ILkEos', 'plus_code': {'compound_code': '6F36+88 Centro, Rio Brilhante - State of Mato Grosso do Sul, Brazil', 'global_code': '58C76F36+88'}, 'rating': 4.6, 'reference': 'ChIJcW5xkbzuiJQRAi8Y7ILkEos'

{'html_attributions': [], 'next_page_token': 'ATtYBwK9Cb0J1GsbHClRqETQHYxmhLcyUhpdXi-jGxJ3GR_rPyqoJQWN5Ltpru4DK-B6MjtKdBYovFSJ__WYM-Ma9WrJ0XmBpZFbOex6SYGMd1IIde6TZpH8cTwif3lHHZDy6v7nfGzAwnoAqgfRzFoDGodfOspZcGjxm2zJ6EBxEJ-hhTkg9oTBvzzMZvx1k9kNiKqcEREmXzlXGY1wgUjJv79cdt9Yqunn1AaH-fjfQK3InYi2F8ohbDEmdosH2bk-J5lDlH1z52k58IA8bMB_DTTgtnXFFhJN6EHstiBG20Ts1j4jUWmkp2GHW8TioWdtW4o4Z8gaDdoffwaMhJJTsAtp8swh7ZhgKaSAZua8_RSUFa90jQYbEgew9ns3zmfUhMyF5RbBixFJ5IDQRSThlgkCpEGlLKMO7dxwqWsJemcuHQYDUC-mQ4Fa0jCTMQ4Bug', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -10.1454044, 'lng': 123.6160471}, 'viewport': {'northeast': {'lat': -10.14402062010728, 'lng': 123.6173753798927}, 'southwest': {'lat': -10.14672027989272, 'lng': 123.6146757201073}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'ASTON Kupang Hotel & Convention Center', 'opening_hours': {'open_now': True}, 'photos': [{'height': 1365, 'html_attributions': ['<a href=

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 16.9864098, 'lng': 73.30778459999999}, 'viewport': {'northeast': {'lat': 16.98778457989272, 'lng': 73.30920142989272}, 'southwest': {'lat': 16.98508492010728, 'lng': 73.30650177010727}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Hotel Landmark', 'photos': [{'height': 3120, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/110486476966272475852">A Google User</a>'], 'photo_reference': 'ATtYBwLj1BdcRM1tpUNqi6GXwie8rHgiHCJOVUPiQyyyaj40qdxxwQjSyJQHMp-6079MZ3kPKyufehHQB30jthp4IU7m671BGK0DPx0id0A9obl0ohIQBYRLmExIKhXyh5HESRgnhcLVU8N4E18Aq2Ci6X_k9mREg8zxRBOzCPa0X_lpfQmU', 'width': 4160}], 'place_id': 'ChIJxYQepGkN6jsRLuEpORV96sI', 'plus_code': {'compound_code': 'X8P5+H4 Ratnagiri, Maharashtra, India', 'global_code': '7J8MX8P5+H4'}, 'rating': 3.8, 'reference': 'ChIJxYQepGkN6jsRLuEpORV96sI', 'scope': 'GOOGLE', 'types': 

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 20.9213625, 'lng': -17.0426394}, 'viewport': {'northeast': {'lat': 20.92283567989272, 'lng': -17.04125457010728}, 'southwest': {'lat': 20.92013602010728, 'lng': -17.04395422989272}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'HOtel Tasiast', 'photos': [{'height': 1960, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/101225533008456979506">A Google User</a>'], 'photo_reference': 'ATtYBwImu5O5X54swzS4daVwhPq6z_KF47a159JMExlyMruuJ55WobkMKvYMaZXTQ3chuVqK94aqVELLCU_gA3WyA0UUu7YX-L15vIRQxiFg2e3esGTyupMwV-gX4UBcrZEB-HUaAa70O0SWY8cCyIm2mXXY7vNxNEfGVylWLEPg79umhWQE', 'width': 4032}], 'place_id': 'ChIJowrumrCzoQ4RGJf6EPWZDJ0', 'plus_code': {'compound_code': 'WXC4+GW Nouadhibou, Mauritania', 'global_code': '7CG4WXC4+GW'}, 'rating': 4.1, 'reference': 'ChIJowrumrCzoQ4RGJf6EPWZDJ0', 'scope': 'GOOGLE', 'types': ['lodging', 

{'html_attributions': [], 'next_page_token': 'ATtYBwJ9-l9cJbA5mgyfLvdWB9-LEknCbpe6ITo0JPuHsULZItaHC2rNuzfWx9yixDB_m3oltowHgPyKKPqB_Ay-8jrGT8IOYZC3UhjmjjvAYl3G7fwZuBl4jEdKu03ByhkghEA1Biu0bjk9_fYC05XAk1U53dg6PtXvYoNt1ftajAHoTkYdPA-kAPLUyQLLNu3W-er7VStrSABwc3SccnIebL_G4COTWvMVotZIIzHT83P29-H3rQyKUvRReHoyd6DgrD__7In3jZHl2EjJd_6f-_jt77HbA7aiwp_e6Zm_1Clu4l4iTAnsfTAFAajhxCCXcLBu6IEjK7N0OARRONEYmX6czVAM1_u34Et_Y9XdUImeku6NN9RodJF0OrsyjOPQpV0RCA9eKJ5DwtmPLFI2VooEUA7sPjeimHkLjGtkugt-SDtqsxLiuH6EqWpGB9RKMQ', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 16.826175, 'lng': -99.86206399999999}, 'viewport': {'northeast': {'lat': 16.82758302989272, 'lng': -99.86074012010728}, 'southwest': {'lat': 16.82488337010728, 'lng': -99.86343977989273}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Hotel Las Brisas Acapulco', 'photos': [{'height': 3456, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/1104121

In [ ]:
cities_pd.to_csv("Hotels.csv")

# Visualize to confirm airport data appears
cities_pd.head(10)

In [ ]:
# Add marker layer ontop of heat map


# Display figure
